In [ ]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import cartopy
import pickle
import xarray as xr
from matplotlib import colors
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import matplotlib as mpl
from scipy.stats import pearsonr

In [ ]:
mpl.__version__

In [ ]:
mpl.rc('font', family='DejaVu Serif') 
# mpl.rc('font', serif='Helvetica Neue') 

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
pinatubo = np.datetime64('1991-06')
agung = np.datetime64('1963-02')
elchichon = np.datetime64('1982-04')

# Pr

In [ ]:
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    solver_list_month = pickle.load(pfile)
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-False-month-False-unforced-False-joint-False', 'rb') as pfile:
    solver_list = pickle.load(pfile)

In [ ]:
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-False-joint-False', 'rb') as pfile:
    solver_list_month_stand = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-False-unforced-False-joint-False', 'rb') as pfile:
    solver_list_stand = pickle.load(pfile)

In [ ]:
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-True-joint-False', 'rb') as pfile:
    solver_list_month_unforced = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-False-unforced-True-joint-False', 'rb') as pfile:
    solver_list_unforced = pickle.load(pfile)

In [ ]:
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/pr-metrics-stand-False-month-False-unforced-False-joint-False', 'rb') as pfile:
    metrics_mswep = pickle.load(pfile)
mswep_pc = metrics_mswep['obs_pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-False-month-False-unforced-False-joint-False', 'rb') as pfile:
    metrics_gpcp = pickle.load(pfile)
gpcp_pc = metrics_gpcp['obs_pc']
pc1 = metrics_gpcp['pc']
pc2 = metrics_mswep['pc']

In [ ]:
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/pr-metrics-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    metrics_mswep_month = pickle.load(pfile)
mswep_pc_month = metrics_mswep_month['obs_pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    metrics_gpcp_month = pickle.load(pfile)
gpcp_pc_month = metrics_gpcp_month['obs_pc']
pc1_month = metrics_gpcp_month['pc']
pc2_month = metrics_mswep_month['pc']

In [ ]:
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-True-month-True-unforced-False-joint-False', 'rb') as pfile:
    metrics_gpcp_month_stand = pickle.load(pfile)
pc1_month_stand = metrics_gpcp_month_stand['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-True-month-False-unforced-False-joint-False', 'rb') as pfile:
    metrics_gpcp_stand = pickle.load(pfile)
pc1_stand = metrics_gpcp_stand['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-True-month-False-unforced-True-joint-False', 'rb') as pfile:
    metrics_gpcp_unforced = pickle.load(pfile)
pc1_unforced = metrics_gpcp_unforced['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-True-month-True-unforced-True-joint-False', 'rb') as pfile:
    metrics_gpcp_month_unforced = pickle.load(pfile)
pc1_month_unforced = metrics_gpcp_month_unforced['pc']

In [ ]:
pc1.time.data[0]

In [ ]:
pc1_month.time.data[0]
pc1_time = pc1_month.indexes['time'].to_datetimeindex()

In [ ]:
def plot_eofs(solver_list_month, solver_list, column, fig, gs, norm, cmap='BrBG', left=True, skip=False):
    eof_all = None
    for i in range(6):
        eof = solver_list_month[i].eofs().isel(mode=0).transpose('lat', 'lon')
        variance = str(solver_list_month[i].varianceFraction().data[0].round(2))
        ax = fig.add_subplot(gs[i, column], projection=cartopy.crs.Robinson(central_longitude=180))
        ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
        ax.set_title(months[i]+' '+variance, 
                    fontsize=12)
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
        if eof_all is None:
            eof_all = eof.data
        else:
            eof_all+=eof.data
    for i in range(6, 12):
        eof = solver_list_month[i].eofs().isel(mode=0).transpose('lat', 'lon')
        variance = str(solver_list_month[i].varianceFraction().data[0].round(2))
        if skip:
            ax = fig.add_subplot(gs[11-i, column+2], projection=cartopy.crs.Robinson(central_longitude=180))
        else:
            ax = fig.add_subplot(gs[11-i, column+1], projection=cartopy.crs.Robinson(central_longitude=180))
        ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
        ax.set_title(months[i]+' '+variance, 
                    fontsize=12)
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
        if eof_all is None:
            eof_all = eof.data
        else:
            eof_all+=eof.data
    '''if left:
        ax = fig.add_subplot(gs[6, column], projection=cartopy.crs.Robinson(central_longitude=180))
    else:
        ax = fig.add_subplot(gs[6, column+1], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    ax.contourf(eof.lon, eof.lat, eof_all/12, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
    ax.set_title('Mean EOF')
    if left:
        ax = fig.add_subplot(gs[7, column], projection=cartopy.crs.Robinson(central_longitude=180))
    else:
        ax = fig.add_subplot(gs[7, column+1], projection=cartopy.crs.Robinson(central_longitude=180))
    
    ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    ax.set_title('AllM '+variance)'''
    eof = solver_list[0].eofs().isel(mode=0).transpose('lat', 'lon')
    variance = str(solver_list[0].varianceFraction().data[0].round(2))
    return eof_all/12, eof, variance

In [ ]:
sns_colors = sns.color_palette('muted', 3)
sns_colors

In [ ]:
pc1.plot()
pc1_stand.plot()

In [ ]:
def plot_eofs_multicols(solver_list_month, solver_list, column, fig, gs, norm, cmap='BrBG', left=True, skip=False):
    eof_all = None
    for i in range(6):
        eof = solver_list_month[i].eofs().isel(mode=0).transpose('lat', 'lon')
        variance = str(int(solver_list_month[i].varianceFraction().data[0].round(2)*100))+'%'
        ax = fig.add_subplot(gs[i, column:column+3], projection=cartopy.crs.Robinson(central_longitude=180))
        ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
        ax.set_title(months[i]+' '+variance, 
                    fontsize=12)
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
        if eof_all is None:
            eof_all = eof.data
        else:
            eof_all+=eof.data
    for i in range(6, 12):
        eof = solver_list_month[i].eofs().isel(mode=0).transpose('lat', 'lon')
        variance = str(int(solver_list_month[i].varianceFraction().data[0].round(2)*100))+'%'
        if skip:
            ax = fig.add_subplot(gs[11-i, column+3:column+6], projection=cartopy.crs.Robinson(central_longitude=180))
        else:
            ax = fig.add_subplot(gs[11-i, column+1], projection=cartopy.crs.Robinson(central_longitude=180))
        ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
        ax.set_title(months[i]+' '+variance, 
                    fontsize=12)
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
        if eof_all is None:
            eof_all = eof.data
        else:
            eof_all+=eof.data
    '''if left:
        ax = fig.add_subplot(gs[6, column], projection=cartopy.crs.Robinson(central_longitude=180))
    else:
        ax = fig.add_subplot(gs[6, column+1], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    ax.contourf(eof.lon, eof.lat, eof_all/12, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
    ax.set_title('Mean EOF')
    if left:
        ax = fig.add_subplot(gs[7, column], projection=cartopy.crs.Robinson(central_longitude=180))
    else:
        ax = fig.add_subplot(gs[7, column+1], projection=cartopy.crs.Robinson(central_longitude=180))
    
    ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    ax.set_title('AllM '+variance)'''
    eof = solver_list[0].eofs().isel(mode=0).transpose('lat', 'lon')
    variance = str(int(solver_list[0].varianceFraction().data[0].round(2)*100))+'%'
    return eof_all/12, eof, variance

In [ ]:
vmin = -0.08
vmax = 0.08
levels=np.linspace(vmin, vmax, 21)
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
fig = plt.figure(figsize=(10, 12))
gs = GridSpec(8, 13, figure=fig, width_ratios=[1, 1, 1,  1, 1, 1,  0.25, 1, 1, 1, 1, 1,1], wspace=0)
ind = 0
mean_eof1, eof_all1, variance1 = plot_eofs_multicols(column=0, solver_list_month=solver_list_month, 
                                                     solver_list=solver_list, norm=norm, 
                                                     gs=gs, fig=fig, left=True, skip=True)
mean_eof2, eof_all2, variance2 = plot_eofs_multicols(column=7, solver_list_month=solver_list_month_stand, 
                                                     solver_list=solver_list_stand, norm=norm, 
                                                     gs=gs, fig=fig, left=False, skip=True)

plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.2)
ax = fig.add_subplot(gs[6:8, 4:9])
ax.plot(pc1_time, pc1, label='AllM Anomaly', color=sns_colors[0])
ax.plot(pc1_time, pc1_month, label='MbyM Anomaly', color=sns_colors[0], linestyle='dotted')
ax.plot(pc1_time, pc1_stand, label='AllM StdAnomaly', color=sns_colors[1])
ax.plot(pc1_time, pc1_month_stand, label='MbyM StdAnomaly', color=sns_colors[1], linestyle='dotted')
# ax.plot(pc1_time, pc1_unforced, label='AllM UnforcedAnomaly', color=sns_colors[2])
# ax.plot(pc1_time, pc1_month_unforced, label='MbyM UnforcedAnomaly', color=sns_colors[2], linestyle='dotted')
ax.axvline(x=pinatubo, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
plt.legend(ncol=2, bbox_to_anchor=(1.05, -.1))
fig.text(x=0.21, y=1., s='Anomaly', fontsize=14, weight='bold')
fig.text(x=0.66, y=1., s='StdAnomaly', fontsize=14, weight='bold')

'''ax = fig.add_subplot(gs[6, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof1, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF')
ax = fig.add_subplot(gs[6, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof2, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF')'''
ax = fig.add_subplot(gs[6:8, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all1, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance1)
ax = fig.add_subplot(gs[6:8, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all2, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance2)
# fig.text(x=0.78, y=1., s='UnforcedAnomaly', fontsize=14, weight='bold')
cbar_ax = fig.add_axes([-0.01, 0.05, 0.01, 0.15])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap='BrBG'), cax=cbar_ax, extend='both')
cbar_ax.yaxis.set_ticks_position('left')
plt.savefig('pr-EOF-pc.png', dpi=300, bbox_inches='tight')

## Unforced

In [ ]:
vmin = -0.08
vmax = 0.08
levels=np.linspace(vmin, vmax, 21)
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
fig = plt.figure(figsize=(10, 12))
gs = GridSpec(8, 13, figure=fig, width_ratios=[1, 1, 1,  1, 1, 1,  0.25, 1, 1, 1, 1, 1,1], wspace=0)
ind = 0
mean_eof1, eof_all1, variance1 = plot_eofs_multicols(column=0, solver_list_month=solver_list_month_stand, 
                                                     solver_list=solver_list_unforced, norm=norm, 
                                                     gs=gs, fig=fig, left=True, skip=True)
mean_eof2, eof_all2, variance2 = plot_eofs_multicols(column=7, solver_list_month=solver_list_month_unforced, 
                                                     solver_list=solver_list_unforced, norm=norm, 
                                                     gs=gs, fig=fig, left=False, skip=True)

plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.2)
ax = fig.add_subplot(gs[6:8, 4:9])
# ax.plot(pc1_time, pc1, label='AllM Anomaly', color=sns_colors[0])
# ax.plot(pc1_time, pc1_month, label='MbyM Anomaly', color=sns_colors[0], linestyle='-.')
ax.plot(pc1_time, pc1_stand, label='AllM StdAnomaly', color=sns_colors[1])
ax.plot(pc1_time, pc1_month_stand, label='MbyM StdAnomaly', color=sns_colors[1], linestyle='dotted')
ax.plot(pc1_time, pc1_unforced, label='AllM UnforcedAnomaly', color=sns_colors[2], alpha=.5)
ax.plot(pc1_time, pc1_month_unforced, label='MbyM UnforcedAnomaly', color=sns_colors[2], alpha=.5, linestyle='dotted')
ax.axvline(x=pinatubo, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
plt.legend(ncol=2, bbox_to_anchor=(1.05, -.1))
fig.text(x=0.21, y=1., s='StdAnomaly', fontsize=14, weight='bold')
fig.text(x=0.66, y=1., s='UnforcedAnomaly', fontsize=14, weight='bold')

'''ax = fig.add_subplot(gs[6, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof1, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF')
ax = fig.add_subplot(gs[6, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof2, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF')'''
ax = fig.add_subplot(gs[6:8, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all1, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance1)
ax = fig.add_subplot(gs[6:8, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all2, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance2)
# fig.text(x=0.78, y=1., s='UnforcedAnomaly', fontsize=14, weight='bold')
cbar_ax = fig.add_axes([-0.01, 0.05, 0.01, 0.15])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap='BrBG'), cax=cbar_ax, extend='both')
cbar_ax.yaxis.set_ticks_position('left')
plt.savefig('pr-EOF-unforced.png', dpi=180, bbox_inches='tight')

In [ ]:
vmin = -0.08
vmax = 0.08
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
cmap='BrBG'
fig = plt.figure(figsize=(14, 10))
gs = GridSpec(7, 6, figure=fig, wspace=0, height_ratios=[1, 1, 1, .1, 1, 1, 1])

ind = 0
eof_all = None
for i in range(6):
    eof = solver_list_month_stand[i].eofs().isel(mode=0).transpose('lat', 'lon')
    variance = str(solver_list_month_stand[i].varianceFraction().data[0].round(2))
    ax = fig.add_subplot(gs[0, i], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
    ax.set_title(months[i]+' '+variance, 
                fontsize=12)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    if eof_all is None:
        eof_all = eof.data
    else:
        eof_all+=eof.data
for i in range(6, 12):
    eof = solver_list_month_stand[i].eofs().isel(mode=0).transpose('lat', 'lon')
    variance = str(solver_list_month_stand[i].varianceFraction().data[0].round(2))
    ax = fig.add_subplot(gs[1, 11-i], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
    ax.set_title(months[i]+' '+variance, 
                fontsize=12)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    if eof_all is None:
        eof_all = eof.data
    else:
        eof_all+=eof.data
eof_mean = eof_all/12
eof = solver_list_stand[0].eofs().isel(mode=0).transpose('lat', 'lon')
variance = str(solver_list_stand[0].varianceFraction().data[0].round(2))
ax = fig.add_subplot(gs[2, 1:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance, 
            fontsize=12)
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax = fig.add_subplot(gs[2, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
ax.contourf(eof.lon, eof.lat, eof_mean, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF', 
            fontsize=12)
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)

ind = 0
eof_all = None
for i in range(6):
    eof = solver_list_month_unforced[i].eofs().isel(mode=0).transpose('lat', 'lon')
    variance = str(solver_list_month_unforced[i].varianceFraction().data[0].round(2))
    ax = fig.add_subplot(gs[4, i], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
    ax.set_title(months[i]+' '+variance, 
                fontsize=12)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    if eof_all is None:
        eof_all = eof.data
    else:
        eof_all+=eof.data
for i in range(6, 12):
    eof = solver_list_month_unforced[i].eofs().isel(mode=0).transpose('lat', 'lon')
    variance = str(solver_list_month_unforced[i].varianceFraction().data[0].round(2))
    ax = fig.add_subplot(gs[5, 11-i], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
    ax.set_title(months[i]+' '+variance, 
                fontsize=12)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    if eof_all is None:
        eof_all = eof.data
    else:
        eof_all+=eof.data
eof_mean = eof_all/12
eof = solver_list_unforced[0].eofs().isel(mode=0).transpose('lat', 'lon')
variance = str(solver_list_unforced[0].varianceFraction().data[0].round(2))
ax = fig.add_subplot(gs[6, 1:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance, 
            fontsize=12)
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax = fig.add_subplot(gs[6, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
ax.contourf(eof.lon, eof.lat, eof_mean, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF', 
            fontsize=12)
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
fig.text(x=0.1, y=.65, s='StdAnomaly', fontsize=14, weight='bold', rotation='vertical')
fig.text(x=0.1, y=.2, s='UnforcedAnomaly', fontsize=14, weight='bold', rotation='vertical')
cbar_ax = fig.add_axes([0.8, 0.12, 0.005, 0.1])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap='BrBG'), cax=cbar_ax, extend='both')
# plt.savefig('pr-EOF-unforced.png', dpi=180, bbox_inches='tight')

## 1, 4, 7, 10

In [ ]:
mode=1
for i, ind in enumerate(range(12)): 
    eof = solver_list_month[ind].eofs().isel(mode=mode).transpose('lat', 'lon')
    variance = str(int(solver_list_month[ind].varianceFraction().data[mode].round(2)*100))+'%'
    print(i, variance)

In [ ]:
def plot_selected_months(solver_list_month, axes, norm, levels, mode=0, cmap='BrBG'):
    months = ['Jan', 'Apr', 'Jul', 'Oct']
    for i, ind in enumerate([0, 3, 6, 9]):
        ax = axes[i]
        eof = solver_list_month[ind].eofs().isel(mode=mode).transpose('lat', 'lon')
        pc = solver_list_month[ind].pcs().isel(mode=mode) # flip sign based on solver pc. 
        n, p = np.polyfit(np.arange(len(pc)), pc, deg=1)
        if n<0:
            print('Inverse?')
            eof = -eof
        variance = str(int(solver_list_month[ind].varianceFraction().data[mode].round(2)*100))+'%'
        ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
        ax.set_title(months[i]+' '+variance)
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)

In [ ]:
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/pr-metrics-stand-False-month-False-unforced-False-joint-False'+'-n_mode-'+str(2), 'rb') as pfile:
    metrics_mswep = pickle.load(pfile)
pc2 = metrics_mswep['pc']
plt.plot(pc2)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/pr-metrics-stand-False-month-True-unforced-False-joint-False'+'-n_mode-'+str(2), 'rb') as pfile:
    metrics_mswep_month = pickle.load(pfile)
pc2_month = metrics_mswep_month['pc']

with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/pr-metrics-stand-True-month-False-unforced-False-joint-False'+'-n_mode-'+str(2), 'rb') as pfile:
    metrics_mswep = pickle.load(pfile)
pc2_stand = metrics_mswep['pc']
plt.plot(pc2_stand)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/pr-metrics-stand-True-month-True-unforced-False-joint-False'+'-n_mode-'+str(2), 'rb') as pfile:
    metrics_mswep_month = pickle.load(pfile)
pc2_month_stand = metrics_mswep_month['pc']

In [ ]:
(solver_list_month[0].eofs().isel(mode=0).transpose('lat', 'lon')).plot()
plt.show()
(solver_list_month_stand[0].eofs().isel(mode=0).transpose('lat', 'lon')).plot()
plt.show()

In [ ]:
vmin = -0.08
vmax = 0.08
levels=np.linspace(vmin, vmax, 21)
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
fig = plt.figure(figsize=(16, 8))
gs = GridSpec(5, 13, figure=fig, width_ratios=[1, 1, .1, 1, 1, 1, 1, 1, 1, 1, 1, .5, 2], wspace=0, height_ratios=[1, 1, .03, 1,  1, ])
# Anomaly case
axes = []
ax = fig.add_subplot(gs[0, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('b',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
axes.append(ax)
ax = fig.add_subplot(gs[0, 5:7], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[0, 7:9], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[0, 9:11], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
plot_selected_months(solver_list_month, axes, norm, levels)
# AllM
eof = solver_list[0].eofs().isel(mode=0).transpose('lat', 'lon')
variance = str(solver_list[0].varianceFraction().data[0].round(2)*100)+'%'
ax = fig.add_subplot(gs[0, 0:2], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('a',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('S-INV '+variance)
# Second EOF
axes = []
ax = fig.add_subplot(gs[1, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('d',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
axes.append(ax)
ax = fig.add_subplot(gs[1, 5:7], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[1, 7:9], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[1, 9:11], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
plot_selected_months(solver_list_month, axes, norm, levels, mode=1)
# AllM
eof = solver_list[0].eofs().isel(mode=1).transpose('lat', 'lon')
variance = str(int(solver_list[0].varianceFraction().data[1].round(2)*100))+'%'
ax = fig.add_subplot(gs[1, 0:2], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('c',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('S-INV '+variance)
# PC
ax = fig.add_subplot(gs[0, -1])
ax.plot(pc1_time, pc1, label='S-INV PC1', color=sns_colors[0])
ax.plot(pc1_time, pc1_month, label='S-VAR PC1', color=sns_colors[0], linestyle='dotted')
ax.axvline(x=pinatubo, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
ax = fig.add_subplot(gs[1, -1])
ax.plot(pc1_time, pc2, label='S-INV PC2', color=sns_colors[1])
ax.plot(pc1_time, pc2_month, label='S-VAR PC2', color=sns_colors[1], linestyle='dotted')
ax.axvline(x=pinatubo, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
ax.annotate('e',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')

# StdAnomaly case
axes = []
ax = fig.add_subplot(gs[3, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('g',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
axes.append(ax)
ax = fig.add_subplot(gs[3, 5:7], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[3, 7:9], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[3, 9:11], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
plot_selected_months(solver_list_month_stand, axes, norm, levels)


eof = solver_list_stand[0].eofs().isel(mode=0).transpose('lat', 'lon')
variance = str(solver_list_stand[0].varianceFraction().data[0].round(2)*100)+'%'
ax = fig.add_subplot(gs[3, 0:2], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('f',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('S-INV '+variance)


axes = []
ax = fig.add_subplot(gs[4, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('i',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
axes.append(ax)
ax = fig.add_subplot(gs[4, 5:7], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[4, 7:9], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[4, 9:11], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
plot_selected_months(solver_list_month_stand, axes, norm, levels, mode=1)



eof = solver_list_stand[0].eofs().isel(mode=1).transpose('lat', 'lon')
variance = str(int(solver_list_stand[0].varianceFraction().data[1].round(2)*100))+'%'
ax = fig.add_subplot(gs[4, 0:2], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('h',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('S-INV '+variance)

'''ax = fig.add_subplot(gs[8, 3:-2])
ax.plot(pc1_time, pc1_stand, label='S-INV PC1', color=sns_colors[0])
ax.plot(pc1_time, pc1_month_stand, label='S-VAR PC1', color=sns_colors[0], linestyle='dotted')
ax.plot(pc1_time, pc2_stand, label='S-INV PC2', color=sns_colors[1])
ax.plot(pc1_time, pc2_month_stand, label='S-VAR PC2', color=sns_colors[1], linestyle='dotted')
ax.axvline(x=pinatubo, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
ax.annotate('j',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
plt.legend(ncol=1, bbox_to_anchor=(1., .7))'''

fig.text(x=.4, y=1, s='RawAnomaly', fontsize=14, weight='bold')
fig.text(x=.4, y=.47, s='StdAnomaly', fontsize=14, weight='bold')
cbar_ax = fig.add_axes([0.9, 0.23, 0.005, 0.15])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap='BrBG'), cax=cbar_ax, extend='both')
fig.subplots_adjust(left=0, right=.87, bottom=0, top=.98)


# plt.savefig('pr-EOF12-main.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
def plot_pc(ax, pc_inv, pc_var, annotation):
    m1, _ = np.polyfit(np.arange(pc1.shape[0]), pc_inv, deg=1)
    m2, _ = np.polyfit(np.arange(pc1.shape[0]), pc_var, deg=1)
    sig1 = "{:.1e}".format(m1)
    sig2 = "{:.1e}".format(m2)
    ax.plot(pc1_time, pc_inv, label='S-INV '+sig1, color=sns_colors[0])
    ax.plot(pc1_time, pc_var, label='S-VAR '+sig2, color=sns_colors[1])
    ax.set_xticks(pc1_time[::150])
    ax.set_xticklabels(ticks)
    ax.set_title('First PC')
    ax.axvline(x=pinatubo, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
    ax.annotate(annotation,  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
    plt.legend(ncol=1, bbox_to_anchor=(1., .7))

In [ ]:
vmin = -0.08
vmax = 0.08
ticks = [i.strftime("%Y-%m") for i in pc1_time[::150]]
levels=np.linspace(vmin, vmax, 21)
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
fig = plt.figure(figsize=(12, 10))
gs = GridSpec(13, 9, figure=fig, width_ratios=[1, 1, .1, 1, 1, 1, 1, .5, 2], wspace=0, height_ratios=[1, 1, 1, 1,1,1,1,1, .3, 1, 1, 1, 1])
# Anomaly case
axes = []
ax = fig.add_subplot(gs[0:2, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('b',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
axes.append(ax)
ax = fig.add_subplot(gs[2:4, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[2:4, 5:7], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[0:2, 5:7], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
plot_selected_months(solver_list_month, axes, norm, levels)
# AllM
eof = solver_list[0].eofs().isel(mode=0).transpose('lat', 'lon')
pc = solver_list[0].pcs().isel(mode=0)
n, p = np.polyfit(np.arange(len(pc)), pc, deg=1)
if n<0:
    print('inverse allM')
    eof = -eof
variance = str(solver_list[0].varianceFraction().data[0].round(2)*100)+'%'
ax = fig.add_subplot(gs[1:3, 0:2], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('a',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('S-INV '+variance)
# Second EOF
axes = []
ax = fig.add_subplot(gs[4:6, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('e',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
axes.append(ax)
ax = fig.add_subplot(gs[6:8, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[6:8, 5:7], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[4:6, 5:7], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
plot_selected_months(solver_list_month, axes, norm, levels, mode=1)
# AllM
eof = solver_list[0].eofs().isel(mode=1).transpose('lat', 'lon')
variance = str(int(solver_list[0].varianceFraction().data[1].round(2)*100))+'%'
ax = fig.add_subplot(gs[5:7, 0:2], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('d',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('S-INV '+variance)
# PC
ax = fig.add_subplot(gs[1:3, -1])
plot_pc(ax, pc1, pc1_month, annotation='c')
ax.set_title('First PC')

ax = fig.add_subplot(gs[5:7, -1])
plot_pc(ax, pc2, pc2_month, annotation='f')
ax.set_title('Second PC')

# StdAnomaly case
axes = []
ax = fig.add_subplot(gs[9:11, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('h',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
axes.append(ax)
ax = fig.add_subplot(gs[11:13, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[11:13, 5:7], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
ax = fig.add_subplot(gs[9:11, 5:7], projection=cartopy.crs.Robinson(central_longitude=180))
axes.append(ax)
plot_selected_months(solver_list_month_stand, axes, norm, levels)


eof = solver_list_stand[0].eofs().isel(mode=0).transpose('lat', 'lon')
variance = str(solver_list_stand[0].varianceFraction().data[0].round(2)*100)+'%'
ax = fig.add_subplot(gs[10:12, 0:2], projection=cartopy.crs.Robinson(central_longitude=180))
ax.annotate('g',  xy=(0, 1.05), xycoords="axes fraction", fontsize=12, weight='bold')
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm, extend='both', levels=levels)
ax.set_title('S-INV '+variance)


ax = fig.add_subplot(gs[10:12, -1])
plot_pc(ax, pc1_stand, pc1_month_stand, annotation='i')
ax.set_title('First PC')


fig.text(x=.4, y=1, s='RawAnomaly', fontsize=14, weight='bold')
fig.text(x=.4, y=.33, s='StdAnomaly', fontsize=14, weight='bold')
cbar_ax = fig.add_axes([0.25, -0.02, 0.25, 0.005])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap='BrBG'), cax=cbar_ax, extend='both', orientation='horizontal')
fig.subplots_adjust(left=0, right=.87, bottom=0, top=.98)

plt.savefig('pr-EOF12-main.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
eof = solver_list[0].eofs().isel(mode=0).transpose('lat', 'lon')
pc = solver_list[0].pcs().isel(mode=0)
n, p = np.polyfit(np.arange(len(pc)), pc, deg=1)
print(n)
pc = solver_list[0].pcs().isel(mode=0).sel(time=slice('1983', '2020'))
n, p = np.polyfit(np.arange(len(pc)), pc, deg=1)
print(n)

In [ ]:
eof.plot()

In [ ]:

for month in range(1, 13):
    pc_month = pc1_month.sel(time=pc2_stand.time.dt.month==month)
    m, b = np.polyfit(np.arange(pc_month.shape[0]), pc_month.data.flatten(), deg=1)
    if m<0:
        pc_month = -pc_month
        print(month, 'flip')
    pc_month.plot()
    

In [ ]:
print(pc2_month.time[0], pc2_month.time[-1])

In [ ]:

for month in range(1, 13):
    pc_month = pc2_month.sel(time=pc2_month.time.dt.month==month)
    m, b = np.polyfit(np.arange(pc_month.shape[0]), pc_month.data.flatten(), deg=1)
    if m<0:
        pc_month = -pc_month
        print(month, 'flip')
    pc_month.plot()
plt.show()

for month in range(1, 13):
    pc_month = pc2_month_stand.sel(time=pc2_month_stand.time.dt.month==month)
    m, b = np.polyfit(np.arange(pc_month.shape[0]), pc_month.data.flatten(), deg=1)
    if m<0:
        pc_month = -pc_month
        print(month, 'flip')
    pc_month.plot()
plt.show()

In [ ]:
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    solver_list_month = pickle.load(pfile)
n_mode = 1
for month in range(1, 13):
    solver = solver_list_month[month-1]
    ori_pc = solver.pcs(npcs=3, pcscaling=0).isel(mode=n_mode-1).sel(time=slice(str(1979)+'-01-01', str(2019+1)+'-01-01'))
    m, b = np.polyfit(np.arange(len(ori_pc)), ori_pc, deg=1)
    if m<0: # flip sign based on solver pc. 
        print(month)

In [ ]:
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    solver_list_month = pickle.load(pfile)
n_mode = 2
for month in range(1, 13):
    solver = solver_list_month[month-1]
    ori_pc = solver.pcs(npcs=3, pcscaling=0).isel(mode=n_mode-1).sel(time=slice(str(1979)+'-01-01', str(2020+1)+'-01-01'))
    m, b = np.polyfit(np.arange(len(ori_pc)), ori_pc, deg=1)
    if m<0: # flip sign based on solver pc. 
        print('whole', month)
    ori_pc = solver.pcs(npcs=3, pcscaling=0).isel(mode=n_mode-1).sel(time=slice(str(1983)+'-01-01', str(2020+1)+'-01-01'))
    m, b = np.polyfit(np.arange(len(ori_pc)), ori_pc, deg=1)
    if m<0: # flip sign based on solver pc. 
        print(month)

In [ ]:
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-False-joint-False', 'rb') as pfile:
    solver_list_month = pickle.load(pfile)
n_mode = 2
for month in range(1, 13):
    solver = solver_list_month[month-1]
    ori_pc = solver.pcs(npcs=3, pcscaling=0).isel(mode=n_mode-1).sel(time=slice(str(1979)+'-01-01', str(2020+1)+'-01-01'))
    m, b = np.polyfit(np.arange(len(ori_pc)), ori_pc, deg=1)
    if m<0: # flip sign based on solver pc. 
        print('whole', month)
    ori_pc = solver.pcs(npcs=3, pcscaling=0).isel(mode=n_mode-1).sel(time=slice(str(1983)+'-01-01', str(2020+1)+'-01-01'))
    m, b = np.polyfit(np.arange(len(ori_pc)), ori_pc, deg=1)
    if m<0: # flip sign based on solver pc. 
        print(month)

# SST

In [ ]:
variable = 'tos'
eof_start = 1950
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    solver_list_month = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-False-month-False-unforced-False-joint-False', 'rb') as pfile:
    solver_list = pickle.load(pfile)

In [ ]:
variable = 'tos'
eof_start = 1950
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-False-joint-False', 'rb') as pfile:
    solver_list_month_stand = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-False-unforced-False-joint-False', 'rb') as pfile:
    solver_list_stand = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-True-joint-False', 'rb') as pfile:
    solver_list_month_unforced = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-False-unforced-True-joint-False', 'rb') as pfile:
    solver_list_unforced = pickle.load(pfile)

In [ ]:
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1950_2022/ERSST/tos-metrics-stand-False-month-False-unforced-False-joint-False', 'rb') as pfile:
    metrics_ersst = pickle.load(pfile)
pc1 = metrics_ersst['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1950_2022/ERSST/tos-metrics-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    metrics_ersst_month = pickle.load(pfile)
pc1_month = metrics_ersst_month['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1950_2022/ERSST/tos-metrics-stand-True-month-True-unforced-False-joint-False', 'rb') as pfile:
    metrics_ersst_month_stand = pickle.load(pfile)
pc1_month_stand = metrics_ersst_month_stand['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1950_2022/ERSST/tos-metrics-stand-True-month-False-unforced-False-joint-False', 'rb') as pfile:
    metrics_ersst_stand = pickle.load(pfile)
pc1_stand = metrics_ersst_stand['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1950_2022/ERSST/tos-metrics-stand-True-month-False-unforced-True-joint-False', 'rb') as pfile:
    metrics_ersst_unforced = pickle.load(pfile)
pc1_unforced = metrics_ersst_unforced['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1950_2022/ERSST/tos-metrics-stand-True-month-True-unforced-True-joint-False', 'rb') as pfile:
    metrics_ersst_month_unforced = pickle.load(pfile)
pc1_month_unforced = metrics_ersst_month_unforced['pc']

In [ ]:
pc1_month.time.data[0]
pc1_time = pc1_month.indexes['time'].to_datetimeindex()
sns_colors = sns.color_palette('muted', 3)
sns_colors

In [ ]:
solver_list_unforced[0].eofs().isel(mode=0).plot()

In [ ]:
vmin = -0.03
vmax = 0.03
levels=np.linspace(vmin, vmax, 21)
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
cmap = 'coolwarm'
fig = plt.figure(figsize=(10, 12))
gs = GridSpec(8, 13, figure=fig, width_ratios=[1, 1, 1,  1, 1, 1,  0.25, 1, 1, 1, 1, 1,1], wspace=0)
ind = 0
mean_eof1, eof_all1, variance1 = plot_eofs_multicols(column=0, solver_list_month=solver_list_month, 
                                                     solver_list=solver_list, norm=norm, cmap=cmap,
                                                     gs=gs, fig=fig, left=True, skip=True)
mean_eof2, eof_all2, variance2 = plot_eofs_multicols(column=7, solver_list_month=solver_list_month_stand, 
                                                     solver_list=solver_list_stand, norm=norm, cmap=cmap,
                                                     gs=gs, fig=fig, left=False, skip=True)

plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.2)
ax = fig.add_subplot(gs[6:8, 4:9])
ax.plot(pc1_time, pc1, label='AllM Anomaly', color=sns_colors[0])
ax.plot(pc1_time, pc1_month, label='MbyM Anomaly', color=sns_colors[0], linestyle='dotted')
ax.plot(pc1_time, pc1_stand, label='AllM StdAnomaly', color=sns_colors[1])
ax.plot(pc1_time, pc1_month_stand, label='MbyM StdAnomaly', color=sns_colors[1], linestyle='dotted')
# ax.plot(pc1_time, pc1_unforced, label='AllM UnforcedAnomaly', color=sns_colors[2])
# ax.plot(pc1_time, pc1_month_unforced, label='MbyM UnforcedAnomaly', color=sns_colors[2], linestyle='-.')
ax.axvline(x=pinatubo, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
ax.axvline(x=elchichon, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
ax.axvline(x=agung, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
plt.legend(ncol=2, bbox_to_anchor=(1.05, -.1))
fig.text(x=0.21, y=1., s='Anomaly', fontsize=14, weight='bold')
fig.text(x=0.66, y=1., s='StdAnomaly', fontsize=14, weight='bold')

'''ax = fig.add_subplot(gs[6, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof1, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF')
ax = fig.add_subplot(gs[6, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof2, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF')'''
ax = fig.add_subplot(gs[6:8, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all1, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance1)
ax = fig.add_subplot(gs[6:8, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all2, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance2)
cbar_ax = fig.add_axes([-0.01, 0.05, 0.01, 0.15])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cbar_ax, extend='both')
cbar_ax.yaxis.set_ticks_position('left')
plt.savefig('tos-EOF-pc.png', dpi=200, bbox_inches='tight')

## Unforced

In [ ]:
vmin = -0.03
vmax = 0.03
levels=np.linspace(vmin, vmax, 21)
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
cmap = 'coolwarm'
fig = plt.figure(figsize=(10, 12))
gs = GridSpec(8, 13, figure=fig, width_ratios=[1, 1, 1,  1, 1, 1,  0.25, 1, 1, 1, 1, 1,1], wspace=0)
ind = 0
mean_eof1, eof_all1, variance1 = plot_eofs_multicols(column=0, solver_list_month=solver_list_month_stand, 
                                                     solver_list=solver_list_stand, norm=norm, cmap=cmap,
                                                     gs=gs, fig=fig, left=True, skip=True)
mean_eof2, eof_all2, variance2 = plot_eofs_multicols(column=7, solver_list_month=solver_list_month_unforced, 
                                                     solver_list=solver_list_unforced, norm=norm, cmap=cmap,
                                                     gs=gs, fig=fig, left=False, skip=True)

plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.2)
ax = fig.add_subplot(gs[6:8, 4:9])
# ax.plot(pc1_time, pc1, label='AllM Anomaly', color=sns_colors[0])
# ax.plot(pc1_time, pc1_month, label='MbyM Anomaly', color=sns_colors[0], linestyle='-.')
ax.plot(pc1_time, pc1_stand, label='AllM StdAnomaly', color=sns_colors[1])
ax.plot(pc1_time, pc1_month_stand, label='MbyM StdAnomaly', color=sns_colors[1], linestyle='dotted')
ax.plot(pc1_time, pc1_unforced, label='AllM UnforcedAnomaly', color=sns_colors[2], alpha=.5)
ax.plot(pc1_time, pc1_month_unforced, label='MbyM UnforcedAnomaly', color=sns_colors[2], alpha=.5, linestyle='dotted')
ax.axvline(x=pinatubo, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
ax.axvline(x=elchichon, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
ax.axvline(x=agung, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
plt.legend(ncol=2, bbox_to_anchor=(1.05, -.1))
fig.text(x=0.21, y=1., s='StdAnomaly', fontsize=14, weight='bold')
fig.text(x=0.66, y=1., s='UnforcedAnomaly', fontsize=14, weight='bold')

'''ax = fig.add_subplot(gs[6, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof1, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF')
ax = fig.add_subplot(gs[6, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof2, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF')'''
ax = fig.add_subplot(gs[6:8, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all1, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance1)
ax = fig.add_subplot(gs[6:8, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all2, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance2)
cbar_ax = fig.add_axes([-0.01, 0.05, 0.01, 0.15])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cbar_ax, extend='both')
cbar_ax.yaxis.set_ticks_position('left')
plt.savefig('tos-EOF-unforced.png', dpi=180, bbox_inches='tight')

In [ ]:
vmin = -0.03
vmax = 0.03
levels=np.linspace(vmin, vmax, 21)
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
cmap='coolwarm'
fig = plt.figure(figsize=(14, 10))
gs = GridSpec(7, 6, figure=fig, wspace=0, height_ratios=[1, 1, 1, .1, 1, 1, 1])

ind = 0
eof_all = None
for i in range(6):
    eof = solver_list_month_stand[i].eofs().isel(mode=0).transpose('lat', 'lon')
    variance = str(solver_list_month_stand[i].varianceFraction().data[0].round(2))
    ax = fig.add_subplot(gs[0, i], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
    ax.set_title(months[i]+' '+variance, 
                fontsize=12)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    if eof_all is None:
        eof_all = eof.data
    else:
        eof_all+=eof.data
for i in range(6, 12):
    eof = solver_list_month_stand[i].eofs().isel(mode=0).transpose('lat', 'lon')
    variance = str(solver_list_month_stand[i].varianceFraction().data[0].round(2))
    ax = fig.add_subplot(gs[1, 11-i], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
    ax.set_title(months[i]+' '+variance, 
                fontsize=12)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    if eof_all is None:
        eof_all = eof.data
    else:
        eof_all+=eof.data
eof_mean = eof_all/12
eof = solver_list_stand[0].eofs().isel(mode=0).transpose('lat', 'lon')
variance = str(solver_list_stand[0].varianceFraction().data[0].round(2))
ax = fig.add_subplot(gs[2, 1:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance, 
            fontsize=12)
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax = fig.add_subplot(gs[2, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
ax.contourf(eof.lon, eof.lat, eof_mean, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF', 
            fontsize=12)
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)

ind = 0
eof_all = None
for i in range(6):
    eof = solver_list_month_unforced[i].eofs().isel(mode=0).transpose('lat', 'lon')
    variance = str(solver_list_month_unforced[i].varianceFraction().data[0].round(2))
    ax = fig.add_subplot(gs[4, i], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
    ax.set_title(months[i]+' '+variance, 
                fontsize=12)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    if eof_all is None:
        eof_all = eof.data
    else:
        eof_all+=eof.data
for i in range(6, 12):
    eof = solver_list_month_unforced[i].eofs().isel(mode=0).transpose('lat', 'lon')
    variance = str(solver_list_month_unforced[i].varianceFraction().data[0].round(2))
    ax = fig.add_subplot(gs[5, 11-i], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
    ax.set_title(months[i]+' '+variance, 
                fontsize=12)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    if eof_all is None:
        eof_all = eof.data
    else:
        eof_all+=eof.data
eof_mean = eof_all/12
eof = solver_list_unforced[0].eofs().isel(mode=0).transpose('lat', 'lon')
variance = str(solver_list_unforced[0].varianceFraction().data[0].round(2))
ax = fig.add_subplot(gs[6, 1:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance, 
            fontsize=12)
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax = fig.add_subplot(gs[6, 3:5], projection=cartopy.crs.Robinson(central_longitude=180))
ax.contourf(eof.lon, eof.lat, eof_mean, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF', 
            fontsize=12)
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
fig.text(x=0.1, y=.65, s='StdAnomaly', fontsize=14, weight='bold', rotation='vertical')
fig.text(x=0.1, y=.2, s='UnforcedAnomaly', fontsize=14, weight='bold', rotation='vertical')
cbar_ax = fig.add_axes([0.8, 0.12, 0.005, 0.1])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cbar_ax, extend='both')
# plt.savefig('tos-EOF-unforced.png', dpi=180, bbox_inches='tight')

# Monmaxpr

In [ ]:
variable = 'monmaxpr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    solver_list_month = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-False-month-False-unforced-False-joint-False', 'rb') as pfile:
    solver_list = pickle.load(pfile)

with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-False-joint-False', 'rb') as pfile:
    solver_list_month_stand = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-False-unforced-False-joint-False', 'rb') as pfile:
    solver_list_stand = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-True-joint-False', 'rb') as pfile:
    solver_list_month_unforced = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-False-unforced-True-joint-False', 'rb') as pfile:
    solver_list_unforced = pickle.load(pfile)

In [ ]:
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/monmaxpr-metrics-stand-False-month-False-unforced-False-joint-False', 'rb') as pfile:
    metric = pickle.load(pfile)
pc1 = metric['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/monmaxpr-metrics-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    metric = pickle.load(pfile)
pc1_month = metric['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/monmaxpr-metrics-stand-True-month-True-unforced-False-joint-False', 'rb') as pfile:
    metric = pickle.load(pfile)
pc1_month_stand = metric['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/monmaxpr-metrics-stand-True-month-False-unforced-False-joint-False', 'rb') as pfile:
    metric = pickle.load(pfile)
pc1_stand = metric['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/monmaxpr-metrics-stand-True-month-False-unforced-True-joint-False', 'rb') as pfile:
    metric = pickle.load(pfile)
pc1_unforced = metric['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/monmaxpr-metrics-stand-True-month-True-unforced-True-joint-False', 'rb') as pfile:
    metric = pickle.load(pfile)
pc1_month_unforced = metric['pc']

In [ ]:
pc1_month.time.data[0]
pc1_time = pc1_month.indexes['time'].to_datetimeindex()
sns_colors = sns.color_palette('muted', 3)
sns_colors

In [ ]:
solver_list_unforced[0].eofs().isel(mode=0).plot()

In [ ]:
vmin = -0.055
vmax = 0.055
levels=np.linspace(vmin, vmax, 21)
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
cmap = 'BrBG'
fig = plt.figure(figsize=(10, 12))
gs = GridSpec(8, 13, figure=fig, width_ratios=[1, 1, 1,  1, 1, 1,  0.25, 1, 1, 1, 1, 1,1], wspace=0)
ind = 0
mean_eof1, eof_all1, variance1 = plot_eofs_multicols(column=0, solver_list_month=solver_list_month, 
                                                     solver_list=solver_list, norm=norm, cmap=cmap,
                                                     gs=gs, fig=fig, left=True, skip=True)
mean_eof2, eof_all2, variance2 = plot_eofs_multicols(column=7, solver_list_month=solver_list_month_stand, 
                                                     solver_list=solver_list_stand, norm=norm, cmap=cmap,
                                                     gs=gs, fig=fig, left=False, skip=True)

plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.2)
ax = fig.add_subplot(gs[6:8, 4:9])
ax.plot(pc1_time, pc1, label='AllM Anomaly', color=sns_colors[0])
ax.plot(pc1_time, pc1_month, label='MbyM Anomaly', color=sns_colors[0], linestyle='dotted')
ax.plot(pc1_time, pc1_stand, label='AllM StdAnomaly', color=sns_colors[1])
ax.plot(pc1_time, pc1_month_stand, label='MbyM StdAnomaly', color=sns_colors[1], linestyle='dotted')
# ax.plot(pc1_time, pc1_unforced, label='AllM UnforcedAnomaly', color=sns_colors[2])
# ax.plot(pc1_time, pc1_month_unforced, label='MbyM UnforcedAnomaly', color=sns_colors[2], linestyle='-.')
ax.axvline(x=pinatubo, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
ax.axvline(x=elchichon, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
plt.legend(ncol=2, bbox_to_anchor=(1.05, -.1))
fig.text(x=0.21, y=1., s='Anomaly', fontsize=14, weight='bold')
fig.text(x=0.66, y=1., s='StdAnomaly', fontsize=14, weight='bold')

'''ax = fig.add_subplot(gs[6, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof1, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF')
ax = fig.add_subplot(gs[6, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof2, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF')'''
ax = fig.add_subplot(gs[6:8, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all1, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance1)
ax = fig.add_subplot(gs[6:8, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all2, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance2)
cbar_ax = fig.add_axes([-0.01, 0.05, 0.01, 0.15])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cbar_ax, extend='both')
cbar_ax.yaxis.set_ticks_position('left')
plt.savefig('monmaxpr-EOF-pc.png', dpi=200, bbox_inches='tight')

## Unforced

In [ ]:
vmin = -0.055
vmax = 0.055
levels=np.linspace(vmin, vmax, 21)
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
cmap = 'BrBG'
fig = plt.figure(figsize=(10, 12))
gs = GridSpec(8, 13, figure=fig, width_ratios=[1, 1, 1,  1, 1, 1,  0.25, 1, 1, 1, 1, 1,1], wspace=0)
ind = 0
mean_eof1, eof_all1, variance1 = plot_eofs_multicols(column=0, solver_list_month=solver_list_month_stand, 
                                                     solver_list=solver_list_stand, norm=norm, cmap=cmap,
                                                     gs=gs, fig=fig, left=True, skip=True)
mean_eof2, eof_all2, variance2 = plot_eofs_multicols(column=7, solver_list_month=solver_list_month_unforced, 
                                                     solver_list=solver_list_unforced, norm=norm, cmap=cmap,
                                                     gs=gs, fig=fig, left=False, skip=True)

plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.2)
ax = fig.add_subplot(gs[6:8, 4:9])
# ax.plot(pc1_time, pc1, label='AllM Anomaly', color=sns_colors[0])
# ax.plot(pc1_time, pc1_month, label='MbyM Anomaly', color=sns_colors[0], linestyle='-.')
ax.plot(pc1_time, pc1_stand, label='AllM StdAnomaly', color=sns_colors[1])
ax.plot(pc1_time, pc1_month_stand, label='MbyM StdAnomaly', color=sns_colors[1], linestyle='dotted')
ax.plot(pc1_time, pc1_unforced, label='AllM UnforcedAnomaly', color=sns_colors[2], alpha=.5)
ax.plot(pc1_time, pc1_month_unforced, label='MbyM UnforcedAnomaly', color=sns_colors[2], alpha=.5, linestyle='dotted')
ax.axvline(x=pinatubo, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
ax.axvline(x=elchichon, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
plt.legend(ncol=2, bbox_to_anchor=(1.05, -.1))
fig.text(x=0.21, y=1., s='StdAnomaly', fontsize=14, weight='bold')
fig.text(x=0.66, y=1., s='UnforcedAnomaly', fontsize=14, weight='bold')

'''ax = fig.add_subplot(gs[6, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof1, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF')
ax = fig.add_subplot(gs[6, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof2, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('Mean EOF')'''
ax = fig.add_subplot(gs[6:8, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all1, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance1)
ax = fig.add_subplot(gs[6:8, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all2, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance2)
cbar_ax = fig.add_axes([-0.01, 0.05, 0.01, 0.15])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cbar_ax, extend='both')
cbar_ax.yaxis.set_ticks_position('left')
plt.savefig('monmaxpr-EOF-unforced.png', dpi=180, bbox_inches='tight')

In [ ]:
vmin = -0.055
vmax = 0.055
levels=np.linspace(vmin, vmax, 21)
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
cmap = 'BrBG'
fig = plt.figure(figsize=(10, 12))
gs = GridSpec(8, 13, figure=fig, width_ratios=[1, 1, 1,  1, 1, 1,  0.25, 1, 1, 1, 1, 1,1], wspace=0)
ind = 0
mean_eof1, eof_all1, variance1 = plot_eofs_multicols(column=0, solver_list_month=solver_list_month_stand, 
                                                     solver_list=solver_list_stand, norm=norm, cmap=cmap,
                                                     gs=gs, fig=fig, left=True, skip=True)
mean_eof2, eof_all2, variance2 = plot_eofs_multicols(column=7, solver_list_month=solver_list_month_unforced, 
                                                     solver_list=solver_list_unforced, norm=norm, cmap=cmap,
                                                     gs=gs, fig=fig, left=False, skip=True)

plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.2)
'''ax = fig.add_subplot(gs[6:8, 4:9])
# ax.plot(pc1_time, pc1, label='AllM Anomaly', color=sns_colors[0])
# ax.plot(pc1_time, pc1_month, label='MbyM Anomaly', color=sns_colors[0], linestyle='-.')
ax.plot(pc1_time, pc1_stand, label='AllM StdAnomaly', color=sns_colors[1])
ax.plot(pc1_time, pc1_month_stand, label='MbyM StdAnomaly', color=sns_colors[1], linestyle='dotted')
ax.plot(pc1_time, pc1_unforced, label='AllM UnforcedAnomaly', color=sns_colors[2], alpha=.5)
ax.plot(pc1_time, pc1_month_unforced, label='MbyM UnforcedAnomaly', color=sns_colors[2], alpha=.5, linestyle='dotted')
ax.axvline(x=pinatubo, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
ax.axvline(x=elchichon, ymin=0, ymax=1, linestyle='dashed', alpha=.3, color='black')
plt.legend(ncol=2, bbox_to_anchor=(1.05, -.1))'''
fig.text(x=0.21, y=1., s='StdAnomaly', fontsize=14, weight='bold')
fig.text(x=0.66, y=1., s='UnforcedAnomaly', fontsize=14, weight='bold')

ax = fig.add_subplot(gs[6:8, 2:6], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all1, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance1)
ax.text(-0.1, 1.15, 'A)', fontsize=16, fontweight='bold', va='top', ha='right')
ax = fig.add_subplot(gs[6:8, 7:11], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all2, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, extend='both', levels=levels)
ax.set_title('AllM '+variance2)
cbar_ax = fig.add_axes([-0.01, 0.05, 0.01, 0.15])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cbar_ax, extend='both')
cbar_ax.yaxis.set_ticks_position('left')
# plt.savefig('monmaxpr-EOF-unforced.png', dpi=180, bbox_inches='tight')